In [52]:
import re
import pandas as pd
from urllib.request import Request, urlopen

## Magnus

In [132]:
df_magnus = pd.DataFrame(columns=['game_id', 
                                  'white_player', 
                                  'white_player_rating',
                                  'black_player',
                                  'black_player_rating',
                                  'game_result',
                                  'move',
                                  'ECO',
                                  'site',
                                  'year'])

n = 0

while True:
    n += 1
    begin = len(df_magnus)
    url=f"https://2700chess.com/games?search=Carlsen%2C+Magnus&page={n}"
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    web_byte = urlopen(req).read()

    webpage = web_byte.decode('utf-8')
            
    result = re.findall('tr data-key(.*)tr', webpage)
            
    for i in result:
        lst = []
        for k in i.split('><'):
            try:
                val_temp = re.search(r'>(.*)<', k).group(1)
                if val_temp == ' ':
                    lst.append(None)
                else:
                    lst.append(val_temp)
            except: 
                pass
        try:
            df_magnus.loc[len(df_magnus)] = lst
        except:
            lst.insert(7, None)
            df_magnus.loc[len(df_magnus)] = lst
    
    if begin == len(df_magnus):
        break
    else:
        pass
    

In [133]:
df_magnus

,game_id,white_player,white_player_rating,black_player,black_player_rating,game_result,move,ECO,site,year
0,1,"Giri, Anish",2764,"Carlsen, Magnus",2859,1-0,35,E15,Wijk aan Zee,2023
1,2,"Carlsen, Magnus",2859,"Ding, Liren",2811,1/2-1/2,37,A46,Wijk aan Zee,2023
2,3,"Keymer, Vincent",2696,"Carlsen, Magnus",2859,0-1,65,D91,Wijk aan Zee,2023
3,4,"Carlsen, Magnus",2859,"Aronian, Levon",2735,1/2-1/2,41,E10,Wijk aan Zee,2023
4,5,"Carlsen, Magnus",2830,"Abdusattorov, Nodirbek",2666,1-0,38,C77,Almaty,2022
...,...,...,...,...,...,...,...,...,...,...
3992,3993,"Kaiser, Guenter",None,"Carlsen, Magnus",None,0-1,36,B08,Bad Wiessee,2000
3993,3994,"Carlsen, Magnus",None,"Cordts, Ingo",2222,0-1,30,A31,Bad Wiessee,2000
3994,3995,"Carlsen, Magnus",2855,"Tari, Aryan",2642,1-0,47,None,C55,http://www.chessbomb.com
3995,3996,"Carlsen, Magnus",2855,"Tari, Aryan",2642,1-0,47,None,C55,http://www.chessbomb.com


In [135]:
df_magnus_csv = df_magnus[:-3].copy()

In [136]:
def chess_result_player(color, game_result):
    game_result = game_result.split('-')
    if color == 'white':
        if game_result[0] == '1':
            return 'win'
        elif game_result[0] == '0':
            return 'lose'
        else:
            return 'draw'
    else:
        if game_result[1] == '1':
            return 'win'
        elif game_result[1] == '0':
            return 'lose'
        else:
            return 'draw'

In [137]:
df_magnus_csv['play_as'] = df_magnus_csv['white_player'].apply(lambda x: 'white' if x == 'Carlsen, Magnus' else 'black')
df_magnus_csv['rating'] = df_magnus_csv.apply(lambda x: x['white_player_rating'] if x['play_as'] == 'white' else x['black_player_rating'], axis=1)
df_magnus_csv['opponent'] = df_magnus_csv.apply(lambda x: x['white_player'] if x['play_as'] == 'black' else x['black_player'], axis=1)
df_magnus_csv['opponent_rating'] = df_magnus_csv.apply(lambda x: x['black_player_rating'] if x['play_as'] == 'white' else x['white_player_rating'], axis=1)
df_magnus_csv['result'] = df_magnus_csv.apply(lambda x: chess_result_player(x['play_as'], x['game_result']), axis=1)



In [138]:
df_magnus_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].head(3)

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2859,black,"Giri, Anish",2764,lose,35,E15,Wijk aan Zee,2023
1,2859,white,"Ding, Liren",2811,draw,37,A46,Wijk aan Zee,2023
2,2859,black,"Keymer, Vincent",2696,win,65,D91,Wijk aan Zee,2023


In [139]:
df_magnus_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].to_csv('magnus.csv', index=False)

In [140]:
df_test = pd.read_csv('magnus.csv')
df_test.head()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2859.0,black,"Giri, Anish",2764.0,lose,35,E15,Wijk aan Zee,2023
1,2859.0,white,"Ding, Liren",2811.0,draw,37,A46,Wijk aan Zee,2023
2,2859.0,black,"Keymer, Vincent",2696.0,win,65,D91,Wijk aan Zee,2023
3,2859.0,white,"Aronian, Levon",2735.0,draw,41,E10,Wijk aan Zee,2023
4,2830.0,white,"Abdusattorov, Nodirbek",2666.0,win,38,C77,Almaty,2022


In [141]:
df_test.tail()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
3989,NaN,black,"Skovgaard, Peter Nicolai",2130.0,win,37,D94,Norway,2001
3990,NaN,black,"Bartels, Holger",2170.0,draw,48,C59,Bad Wiessee,2000
3991,NaN,white,"Olzem, Lothar",2179.0,draw,36,D00,Bad Wiessee,2000
3992,NaN,black,"Kaiser, Guenter",NaN,win,36,B08,Bad Wiessee,2000
3993,NaN,white,"Cordts, Ingo",2222.0,lose,30,A31,Bad Wiessee,2000


## Kasparov


In [142]:
df_garry = pd.DataFrame(columns=['game_id', 
                                  'white_player', 
                                  'white_player_rating',
                                  'black_player',
                                  'black_player_rating',
                                  'game_result',
                                  'move',
                                  'ECO',
                                  'site',
                                  'year'])

n = 0

while True:
    n += 1
    begin = len(df_garry)
    url=f"https://2700chess.com/games?search=Kasparov,%20Garry&page={n}"
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    web_byte = urlopen(req).read()

    webpage = web_byte.decode('utf-8')
            
    result = re.findall('tr data-key(.*)tr', webpage)
            
    for i in result:
        lst = []
        for k in i.split('><'):
            try:
                val_temp = re.search(r'>(.*)<', k).group(1)
                if val_temp == ' ':
                    lst.append(None)
                else:
                    lst.append(val_temp)
            except: 
                pass
        try:
            df_garry.loc[len(df_garry)] = lst
        except:
            lst.insert(7, None)
            df_garry.loc[len(df_garry)] = lst
    
    if begin == len(df_garry):
        break
    else:
        pass
    

In [143]:
df_garry

,game_id,white_player,white_player_rating,black_player,black_player_rating,game_result,move,ECO,site,year
0,1,"Nepomniachtchi, Ian",2807,"Kasparov, Garry",2801,1/2-1/2,30,A10,Zagreb,2021
1,2,"Kasparov, Garry",2801,"Vachier-Lagrave, Maxime",2794,0-1,43,A04,Zagreb,2021
2,3,"Giri, Anish",2744,"Kasparov, Garry",2801,1/2-1/2,39,A04,Zagreb,2021
3,4,"Kasparov, Garry",2801,"Mamedyarov, Shakhriyar",2716,0-1,26,D61,Zagreb,2021
4,5,"Korobov, Anton",2668,"Kasparov, Garry",2801,1-0,17,E73,Zagreb,2021
...,...,...,...,...,...,...,...,...,...,...
2577,2578,"Kasparov, Garry",None,"Averbakh, Yuri L",None,1-0,48,C68,Moscow,1974
2578,2579,"Magerramov, Elmar",None,"Kasparov, Garry",None,0-1,35,B84,Baku,1973
2579,2580,"Kengis, Edvins",None,"Kasparov, Garry",None,1/2-1/2,54,B89,Vilnius,1973
2580,2581,"Kasparov, Garry",None,"Muratkuliev, Shohrat",None,1-0,32,C84,Baku,1973


In [154]:
df_garry_csv = df_garry.copy()

In [155]:
def chess_result_player(color, game_result):
    game_result = game_result.split('-')
    if color == 'white':
        if game_result[0] == '1':
            return 'win'
        elif game_result[0] == '0':
            return 'lose'
        else:
            return 'draw'
    else:
        if game_result[1] == '1':
            return 'win'
        elif game_result[1] == '0':
            return 'lose'
        else:
            return 'draw'

In [159]:
df_garry_csv['play_as'] = df_garry_csv['white_player'].apply(lambda x: 'white' if x == 'Kasparov, Garry' else 'black')
df_garry_csv['rating'] = df_garry_csv.apply(lambda x: x['white_player_rating'] if x['play_as'] == 'white' else x['black_player_rating'], axis=1)
df_garry_csv['opponent'] = df_garry_csv.apply(lambda x: x['white_player'] if x['play_as'] == 'black' else x['black_player'], axis=1)
df_garry_csv['opponent_rating'] = df_garry_csv.apply(lambda x: x['black_player_rating'] if x['play_as'] == 'white' else x['white_player_rating'], axis=1)
df_garry_csv['result'] = df_garry_csv.apply(lambda x: chess_result_player(x['play_as'], x['game_result']), axis=1)



In [168]:
df_garry_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].head(5)

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2801,black,"Nepomniachtchi, Ian",2807,draw,30,A10,Zagreb,2021
1,2801,white,"Vachier-Lagrave, Maxime",2794,lose,43,A04,Zagreb,2021
2,2801,black,"Giri, Anish",2744,draw,39,A04,Zagreb,2021
3,2801,white,"Mamedyarov, Shakhriyar",2716,lose,26,D61,Zagreb,2021
4,2801,black,"Korobov, Anton",2668,lose,17,E73,Zagreb,2021


In [169]:
df_garry_csv[['rating', 'play_as', 'opponent', 'opponent_rating', 'result', 'move', 'ECO', 'site', 'year']].to_csv('garry.csv', index=False)

In [170]:
df_test = pd.read_csv('garry.csv')
df_test.head()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
0,2801.0,black,"Nepomniachtchi, Ian",2807.0,draw,30,A10,Zagreb,2021
1,2801.0,white,"Vachier-Lagrave, Maxime",2794.0,lose,43,A04,Zagreb,2021
2,2801.0,black,"Giri, Anish",2744.0,draw,39,A04,Zagreb,2021
3,2801.0,white,"Mamedyarov, Shakhriyar",2716.0,lose,26,D61,Zagreb,2021
4,2801.0,black,"Korobov, Anton",2668.0,lose,17,E73,Zagreb,2021


In [171]:
df_test.tail()

,rating,play_as,opponent,opponent_rating,result,move,ECO,site,year
2577,NaN,white,"Averbakh, Yuri L",NaN,win,48,C68,Moscow,1974
2578,NaN,black,"Magerramov, Elmar",NaN,win,35,B84,Baku,1973
2579,NaN,black,"Kengis, Edvins",NaN,draw,54,B89,Vilnius,1973
2580,NaN,white,"Muratkuliev, Shohrat",NaN,win,32,C84,Baku,1973
2581,NaN,white,Vasilienko,NaN,win,40,C04,Kiev,1973
